In [ ]:
import numpy as np
import pandas as pd
import os
import random

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('../input/chinese-mnist-digit-recognizer/chineseMNIST.csv')
data.head()

In [ ]:
print(data.shape)

In [ ]:
label0=data['label'].unique()
Name=sorted(label0)
n=len(label0)

num=[]
for i in range(n):
    num+=[i]

normal_mapping=dict(zip(Name,num)) 
reverse_mapping=dict(zip(num,Name)) 

def mapper(value):
    return reverse_mapping[value]

In [ ]:
normal_mapping

In [ ]:
datax = data.iloc[:,0:4096]
datay = data['label'].map(normal_mapping)

In [ ]:
datax=np.array(datax)
datay=np.array(datay)

In [ ]:
n=len(datax)
N=[]
for i in range(n):
    N+=[i]
random.shuffle(N)

In [ ]:
train=datax[N[0:(n//10)*8]]
test=datax[N[(n//10)*8:]]
trainy=datay[N[0:(n//10)*8]]
testy=datay[N[(n//10)*8:]]

In [ ]:
X=train
y0=trainy

In [ ]:
binencoder = LabelBinarizer()
y = binencoder.fit_transform(y0)
y

In [ ]:
X_images = X.reshape(-1,64,64)
test_images = test.reshape(-1,64,64)

print(X_images.shape)
print(test_images.shape)

In [ ]:
fig,axs = plt.subplots(3,3,figsize=(14,14))
for i in range(9):
    r=i//3
    c=i%3
    ax=axs[r][c].imshow(X_images[i])
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_images, y, test_size = 0.2, random_state=90)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train = X_train/255
X_test = X_test/255

X_train = X_train.reshape(-1,64,64,1).astype('float32')
X_test = X_test.reshape(-1,64,64,1).astype('float32')

In [ ]:
test_images2 = test_images/255
test = test_images2.reshape(-1,64,64,1).astype('float32')

In [ ]:
conv_model = Sequential()

conv_model.add(Conv2D(32,(4,4),input_shape = (64,64,1),activation = 'relu'))
conv_model.add(MaxPooling2D(pool_size=(2,2)))
conv_model.add(Conv2D(64,(3,3),activation = 'relu'))
conv_model.add(MaxPooling2D(pool_size=(2,2)))
conv_model.add(Dropout(0.2))
conv_model.add(Flatten())
conv_model.add(Dense(128,activation='relu'))
conv_model.add(Dense(50, activation='relu'))
conv_model.add(Dense(15, activation='softmax'))
conv_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
conv_model.summary()

In [ ]:
result = conv_model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=64, verbose=2)
result

In [ ]:
history_df = pd.DataFrame(result.history)
history_df.loc[:, ['accuracy', 'val_accuracy']].plot()
history_df.loc[:, ['loss', 'val_loss']].plot()

In [ ]:
pred = conv_model.predict(test)

In [ ]:
pred2=[]
for item in pred:
    value2=np.argmax(item)      
    pred2+=[value2]
pred2[0:10]

In [ ]:
testy2=list(testy)
testy2[0:10]

In [ ]:
accuracy=accuracy_score(testy2,pred2)
print(accuracy)